<a href="https://colab.research.google.com/github/hamzautd7/CE888-Assignment-1/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('bank-additional-full.csv', delimiter=';')

In [3]:
df.shape

(41188, 21)

In [4]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [5]:
df[df['y'] == 'yes'] 

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
75,41,blue-collar,divorced,basic.4y,unknown,yes,no,telephone,may,mon,1575,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
83,49,entrepreneur,married,university.degree,unknown,yes,no,telephone,may,mon,1042,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
88,49,technician,married,basic.9y,no,no,no,telephone,may,mon,1467,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
129,41,technician,married,professional.course,unknown,yes,no,telephone,may,mon,579,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
139,45,blue-collar,married,basic.9y,unknown,yes,no,telephone,may,mon,461,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41174,62,retired,married,university.degree,no,yes,no,cellular,nov,thu,208,1,1,6,success,-1.1,94.767,-50.8,1.031,4963.6,yes
41178,62,retired,married,university.degree,no,no,no,cellular,nov,thu,483,2,6,3,success,-1.1,94.767,-50.8,1.031,4963.6,yes
41181,37,admin.,married,university.degree,no,yes,no,cellular,nov,fri,281,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [6]:
df.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [7]:
df.drop('duration',axis=1, inplace=True)

In [8]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [9]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [10]:
df.nunique()

age                78
job                12
marital             4
education           8
default             3
housing             3
loan                3
contact             2
month              10
day_of_week         5
campaign           42
pdays              27
previous            8
poutcome            3
emp.var.rate       10
cons.price.idx     26
cons.conf.idx      26
euribor3m         316
nr.employed        11
y                   2
dtype: int64

In [11]:
df[["job"]] = df[["job"]].apply(LabelEncoder().fit_transform)
df[["marital"]] = df[["marital"]].apply(LabelEncoder().fit_transform)
df[["education"]] = df[["education"]].apply(LabelEncoder().fit_transform)
df[["default"]] = df[["default"]].apply(LabelEncoder().fit_transform)
df[["loan"]] = df[["loan"]].apply(LabelEncoder().fit_transform)
df[["month"]] = df[["month"]].apply(LabelEncoder().fit_transform)
df[["day_of_week"]] = df[["day_of_week"]].apply(LabelEncoder().fit_transform)
df[["previous"]] = df[["previous"]].apply(LabelEncoder().fit_transform)
df[["poutcome"]] = df[["poutcome"]].apply(LabelEncoder().fit_transform)
df[["emp.var.rate"]] = df[["emp.var.rate"]].apply(LabelEncoder().fit_transform)
df[["nr.employed"]] = df[["nr.employed"]].apply(LabelEncoder().fit_transform)



In [12]:
df[["housing"]] = df[["housing"]].apply(LabelEncoder().fit_transform)
df[["contact"]] = df[["contact"]].apply(LabelEncoder().fit_transform)


In [13]:
cat_features = ['job', 'marital', 'education', 'default', 'housing', 'loan',
        'month', 'day_of_week',  'previous',
       'poutcome', 'emp.var.rate','nr.employed', 'contact']

In [14]:
df = pd.get_dummies(df, columns=cat_features)

In [15]:
df.head()

,age,campaign,pdays,cons.price.idx,cons.conf.idx,euribor3m,y,job_0,job_1,job_2,job_3,job_4,job_5,job_6,job_7,job_8,job_9,job_10,job_11,marital_0,marital_1,marital_2,marital_3,education_0,education_1,education_2,education_3,education_4,education_5,education_6,education_7,default_0,default_1,default_2,housing_0,housing_1,housing_2,loan_0,loan_1,loan_2,...,month_9,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,previous_0,previous_1,previous_2,previous_3,previous_4,previous_5,previous_6,previous_7,poutcome_0,poutcome_1,poutcome_2,emp.var.rate_0,emp.var.rate_1,emp.var.rate_2,emp.var.rate_3,emp.var.rate_4,emp.var.rate_5,emp.var.rate_6,emp.var.rate_7,emp.var.rate_8,emp.var.rate_9,nr.employed_0,nr.employed_1,nr.employed_2,nr.employed_3,nr.employed_4,nr.employed_5,nr.employed_6,nr.employed_7,nr.employed_8,nr.employed_9,nr.employed_10,contact_0,contact_1
0,56,1,999,93.994,-36.4,4.857,no,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,57,1,999,93.994,-36.4,4.857,no,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,37,1,999,93.994,-36.4,4.857,no,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,40,1,999,93.994,-36.4,4.857,no,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,56,1,999,93.994,-36.4,4.857,no,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [16]:
df.shape

(41188, 89)

In [17]:

# dividing the datasets into two parts i.e. training datasets and test datasets 
X = df.drop('y',axis=1)
y = df['y'] 
  
# Spliting arrays or matrices into random train and test subsets 
from sklearn.model_selection import train_test_split 
# i.e. 80 % training dataset and 20 % test datasets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify = y) 

In [18]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((32950, 88), (8238, 88), (32950,), (8238,))

In [39]:
# creating a RF classifier 
clf = RandomForestClassifier(n_estimators = 100,max_features='log2',min_samples_leaf=3, random_state=50)   
  
# Training the model on the training dataset 
# fit function is used to train the model using the training sets as parameters 
clf.fit(X_train, y_train) 
  
# performing predictions on the test dataset 
y_pred = clf.predict(X_test) 
  
# metrics are used to find accuracy or error 
from sklearn import metrics   
print() 
  
# using metrics module for accuracy calculation 
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))


ACCURACY OF THE MODEL:  0.898761835396941


In [20]:
features = ['age', 'campaign', 'pdays', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'job_0', 'job_1', 'job_2', 'job_3', 'job_4', 'job_5',
       'job_6', 'job_7', 'job_8', 'job_9', 'job_10', 'job_11', 'marital_0',
       'marital_1', 'marital_2', 'marital_3', 'education_0', 'education_1',
       'education_2', 'education_3', 'education_4', 'education_5',
       'education_6', 'education_7', 'default_0', 'default_1', 'default_2',
       'housing_0', 'housing_1', 'housing_2', 'loan_0', 'loan_1', 'loan_2',
       'month_0', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'day_of_week_0',
       'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4',
       'previous_0', 'previous_1', 'previous_2', 'previous_3', 'previous_4',
       'previous_5', 'previous_6', 'previous_7', 'poutcome_0', 'poutcome_1',
       'poutcome_2', 'emp.var.rate_0', 'emp.var.rate_1', 'emp.var.rate_2',
       'emp.var.rate_3', 'emp.var.rate_4', 'emp.var.rate_5', 'emp.var.rate_6',
       'emp.var.rate_7', 'emp.var.rate_8', 'emp.var.rate_9', 'nr.employed_0',
       'nr.employed_1', 'nr.employed_2', 'nr.employed_3', 'nr.employed_4',
       'nr.employed_5', 'nr.employed_6', 'nr.employed_7', 'nr.employed_8',
       'nr.employed_9', 'nr.employed_10', 'contact_0', 'contact_1']

In [21]:
feature_imp = pd.Series(clf.feature_importances_, index = features).sort_values(ascending = False) 
print(feature_imp)

age               1.587754e-01
euribor3m         1.329434e-01
campaign          8.201293e-02
pdays             3.244298e-02
cons.conf.idx     2.836254e-02
                      ...     
previous_6        7.765799e-05
previous_7        4.888851e-05
nr.employed_7     3.353077e-05
emp.var.rate_6    9.302634e-06
default_2         4.292448e-07
Length: 88, dtype: float64


In [28]:
type(feature_imp[1])

numpy.float64

In [29]:
from sklearn.model_selection import GridSearchCV

In [32]:
parameters = [{'n_estimators': [80, 100,120,140], 
               'min_samples_leaf': [1,3,5],
               'max_features': [0.5, 'auto', 'sqrt', 'log2'],
               },
              ]

In [33]:
gridSearch = GridSearchCV(estimator=clf, param_grid=parameters, cv=10, n_jobs=-1)
%time gridSearch = gridSearch.fit(X_train,y_train)

bestAcc = gridSearch.best_score_
bestParams = gridSearch.best_params_
bestParams

CPU times: user 8.12 s, sys: 758 ms, total: 8.87 s
Wall time: 32min 5s


{'max_features': 'log2', 'min_samples_leaf': 3, 'n_estimators': 80}

In [34]:
a=1